In [1]:
from pyspark.sql import SparkSession

app = SparkSession.builder.appName("filtering data").getOrCreate()

csv_path = r"C:\Users\ASUS\pyspark\employee_data.csv"

df = app.read.csv(path=csv_path, header=True, inferSchema=True)

df.printSchema()

df.show(n=5)

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: integer (nullable = true)
 |-- department: string (nullable = true)
 |-- region: string (nullable = true)

+---+-------+---+------+----------+------+
| id|   name|age|salary|department|region|
+---+-------+---+------+----------+------+
|  1|  Alice| 25| 50000|        HR| North|
|  2|    Bob| 30| 60000|   Finance| South|
|  3|Charlie| 35| 70000|        IT|  East|
|  4|  David| 40| 80000| Marketing|  West|
|  5|    Eve| 45| 90000|        HR| North|
+---+-------+---+------+----------+------+
only showing top 5 rows



In [3]:
df_filtered = df.filter(df['salary'] > 80000)
df_filtered.show()

+---+-----+---+------+----------+------+
| id| name|age|salary|department|region|
+---+-----+---+------+----------+------+
|  5|  Eve| 45| 90000|        HR| North|
|  6|Frank| 50|100000|   Finance| South|
|  7|Grace| 55|110000|        IT|  East|
|  8| Hank| 60|120000| Marketing|  West|
|  9|  Ivy| 65|130000|        HR| North|
| 10| Jack| 70|140000|   Finance| South|
+---+-----+---+------+----------+------+



In [4]:
df_filt2 = df.groupBy('department').avg('salary')
df_filt2.show()

+----------+-----------+
|department|avg(salary)|
+----------+-----------+
|        HR|    90000.0|
|   Finance|   100000.0|
| Marketing|   100000.0|
|        IT|    90000.0|
+----------+-----------+



In [8]:
df_filt3 = df.groupBy('department').agg({'salary':'avg', 'id':'count'})
df_filt3.show()

+----------+---------+-----------+
|department|count(id)|avg(salary)|
+----------+---------+-----------+
|        HR|        3|    90000.0|
|   Finance|        3|   100000.0|
| Marketing|        2|   100000.0|
|        IT|        2|    90000.0|
+----------+---------+-----------+



In [9]:
# Sort by salary in descending order
sorted_df = df.orderBy(df["salary"], ascending=False)
sorted_df.show()

+---+-------+---+------+----------+------+
| id|   name|age|salary|department|region|
+---+-------+---+------+----------+------+
| 10|   Jack| 70|140000|   Finance| South|
|  9|    Ivy| 65|130000|        HR| North|
|  8|   Hank| 60|120000| Marketing|  West|
|  7|  Grace| 55|110000|        IT|  East|
|  6|  Frank| 50|100000|   Finance| South|
|  5|    Eve| 45| 90000|        HR| North|
|  4|  David| 40| 80000| Marketing|  West|
|  3|Charlie| 35| 70000|        IT|  East|
|  2|    Bob| 30| 60000|   Finance| South|
|  1|  Alice| 25| 50000|        HR| North|
+---+-------+---+------+----------+------+



In [11]:
# Sort by salary in ascending order
sorted_df = df.orderBy(df["salary"], ascending=True)
sorted_df.show()

+---+-------+---+------+----------+------+
| id|   name|age|salary|department|region|
+---+-------+---+------+----------+------+
|  1|  Alice| 25| 50000|        HR| North|
|  2|    Bob| 30| 60000|   Finance| South|
|  3|Charlie| 35| 70000|        IT|  East|
|  4|  David| 40| 80000| Marketing|  West|
|  5|    Eve| 45| 90000|        HR| North|
|  6|  Frank| 50|100000|   Finance| South|
|  7|  Grace| 55|110000|        IT|  East|
|  8|   Hank| 60|120000| Marketing|  West|
|  9|    Ivy| 65|130000|        HR| North|
| 10|   Jack| 70|140000|   Finance| South|
+---+-------+---+------+----------+------+



In [16]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Define the window specification (partition by department, ordered by salary)
windowSpec = Window.partitionBy("department").orderBy("salary")

# Create a new column for rank based on salary within each department
df_with_rank = df.withColumn("rank", F.dense_rank().over(windowSpec))
df_with_rank.show()

+---+-------+---+------+----------+------+----+
| id|   name|age|salary|department|region|rank|
+---+-------+---+------+----------+------+----+
|  2|    Bob| 30| 60000|   Finance| South|   1|
|  6|  Frank| 50|100000|   Finance| South|   2|
| 10|   Jack| 70|140000|   Finance| South|   3|
|  1|  Alice| 25| 50000|        HR| North|   1|
|  5|    Eve| 45| 90000|        HR| North|   2|
|  9|    Ivy| 65|130000|        HR| North|   3|
|  3|Charlie| 35| 70000|        IT|  East|   1|
|  7|  Grace| 55|110000|        IT|  East|   2|
|  4|  David| 40| 80000| Marketing|  West|   1|
|  8|   Hank| 60|120000| Marketing|  West|   2|
+---+-------+---+------+----------+------+----+



In [20]:
windowspec2 = Window.partitionBy('department').orderBy('id')
df_with_row_num = df.withColumn("row_num", F.row_number().over(windowspec2))
df_with_row_num.show()

+---+-------+---+------+----------+------+-------+
| id|   name|age|salary|department|region|row_num|
+---+-------+---+------+----------+------+-------+
|  2|    Bob| 30| 60000|   Finance| South|      1|
|  6|  Frank| 50|100000|   Finance| South|      2|
| 10|   Jack| 70|140000|   Finance| South|      3|
|  1|  Alice| 25| 50000|        HR| North|      1|
|  5|    Eve| 45| 90000|        HR| North|      2|
|  9|    Ivy| 65|130000|        HR| North|      3|
|  3|Charlie| 35| 70000|        IT|  East|      1|
|  7|  Grace| 55|110000|        IT|  East|      2|
|  4|  David| 40| 80000| Marketing|  West|      1|
|  8|   Hank| 60|120000| Marketing|  West|      2|
+---+-------+---+------+----------+------+-------+



In [23]:
# Define Window Specification for each window function
window_spec = Window.partitionBy("department").orderBy("salary")
window_spec_cumsum = Window.partitionBy("department").orderBy("salary").rowsBetween(Window.unboundedPreceding, Window.currentRow)
window_spec_avg = Window.partitionBy("department")

# Apply multiple window functions
df_with_windows = df \
    .withColumn("rank", F.rank().over(window_spec)) \
    .withColumn("cumulative_sum", F.sum("salary").over(window_spec_cumsum)) \
    .withColumn("avg_salary", F.avg("salary").over(window_spec_avg)) \
    .withColumn("max_salary", F.max("salary").over(window_spec_avg)) \
    .withColumn("min_salary", F.min("salary").over(window_spec_avg))

df_with_windows.show()

+---+-------+---+------+----------+------+----+--------------+----------+----------+----------+
| id|   name|age|salary|department|region|rank|cumulative_sum|avg_salary|max_salary|min_salary|
+---+-------+---+------+----------+------+----+--------------+----------+----------+----------+
|  2|    Bob| 30| 60000|   Finance| South|   1|         60000|  100000.0|    140000|     60000|
|  6|  Frank| 50|100000|   Finance| South|   2|        160000|  100000.0|    140000|     60000|
| 10|   Jack| 70|140000|   Finance| South|   3|        300000|  100000.0|    140000|     60000|
|  1|  Alice| 25| 50000|        HR| North|   1|         50000|   90000.0|    130000|     50000|
|  5|    Eve| 45| 90000|        HR| North|   2|        140000|   90000.0|    130000|     50000|
|  9|    Ivy| 65|130000|        HR| North|   3|        270000|   90000.0|    130000|     50000|
|  3|Charlie| 35| 70000|        IT|  East|   1|         70000|   90000.0|    110000|     70000|
|  7|  Grace| 55|110000|        IT|  Eas